In [8]:
import glob
import pickle
import numpy as np
import pandas as pd
from astropy.coordinates import SkyCoord
import astropy.units as u

In [10]:
def f1(x):
    return '%5.2f' % x

In [2]:
cd ~/Documents/HERMES/reductions/6.5/m67_lr/obj_single/

/Users/Carlos/Documents/HERMES/reductions/6.5/m67_lr/obj_single


In [3]:
filehandler = open('M67-375.obj', 'r')
thisStar = pickle.load(filehandler)
thisCamera= thisStar.exposures.cameras[0]
thisCamera.fileNames
thisStar.exposures.plates[10][-1]
# print np.vstack((thisStar.exposures.pivots,np.nansum(thisCamera.red_fluxes,axis=1))).transpose()
# print thisStar.exposures.p

'0'

In [4]:

outFileNames = np.load('/Users/Carlos/Documents/ipn/npy/outFileNames.npy')
# outObjects = np.load('npy/outObjects.npy')
# outPlates = np.load('npy/outPlates.npy')
outMJD_Exp = np.load('/Users/Carlos/Documents/ipn/npy/outMJD_Exp.npy')

outFileNames = np.delete(outFileNames,86)
# outObjects = np.delete(outObjects,86)
# outPlates = np.delete(outPlates,86)
outMJD_Exp = np.delete(outMJD_Exp,86, axis = 0)

ordIdx = np.argsort(outMJD_Exp[:,0])

outFileNames = outFileNames[ordIdx]
outMJD_Exp = outMJD_Exp[ordIdx]



In [11]:
obsMatrix = np.ones((292, 99, 2, 4))*np.nan #stars, observations, [pivot, plate], cam

objList = glob.glob('*.obj')

outArray = []
i = 0
for thisObj in objList:
    if 'red' not in thisObj:
        filehandler = open(thisObj, 'r')
        thisStar = pickle.load(filehandler)
#         print thisStar.name
        
        gc = SkyCoord(ra=thisStar.RA*u.si.degree, dec=thisStar.Dec*u.si.degree)
        name = thisStar.name
        mag = thisStar.Vmag
        RAhms = str(int(gc.ra.hms.h))+'h'+str(int(gc.ra.hms.m))+'m'+str(f1(gc.ra.hms.s))+'s'
        RAdms = str(gc.ra)
        RAdeg = str(gc.ra.deg)
        DECdms = str(gc.dec)
        DECdeg = str(gc.dec.deg)
#         pivot = a[1].data['PIVOT'][i]
        i += 1
        ID = i
        outArray.append((ID, name, mag, RAhms, RAdms, RAdeg, DECdms, DECdeg))
        
        for cam in range(4):
#             print 'Camera',cam,
            thisCam = thisStar.exposures.cameras[cam]

            for exp in range(len(thisCam.fileNames)):
                if np.nansum(thisCam.red_fluxes[exp])!=0:
                    thisFileName = thisStar.exposures.cameras[0].fileNames[exp][:10]+'.fits'
#                     print thisFileName
                    fileIdx = np.where(outFileNames==thisFileName)[0]
                    if len(fileIdx)>0:
                        fileIdx = fileIdx[0]
                        obsMatrix[ID-1, fileIdx, 0, cam] = thisStar.exposures.pivots[exp]
                        obsMatrix[ID-1, fileIdx, 1, cam] = thisStar.exposures.plates[exp][-1]
#                         print 'Found', ID, fileIdx, cam
            
            
            
            
        filehandler.close()
        thisStar = None

outArray = np.array(outArray)
np.save('outArray.npy',outArray)

labels = ['ID', 'Name','Vmag','RA(hms)','RA(dms)','RA(deg)','DEC(dms)','DEC(deg)']
df = pd.DataFrame(outArray)
df.columns = labels

outLatex = df.to_latex(index=False)
print df.to_latex(index=False)

\begin{tabular}{llllllll}
\toprule
  ID &          Name &    Kmag &      RA(hms) &         RA(dms) &        RA(deg) &      DEC(dms) &       DEC(deg) \\
\midrule
   1 &       M67-375 &   15.81 &  8h50m45.62s &    132d41m24.3s &  132.690083333 &     11d55m36s &  11.9266666667 \\
   2 &       M67-378 &  14.788 &  8h50m46.86s &   132d41m42.84s &  132.695233333 &  11d52m57.85s &  11.8827361111 \\
   3 &       M67-381 &    13.9 &  8h50m47.62s &    132d41m54.3s &  132.698416667 &   11d45m23.2s &  11.7564444444 \\
   4 &       M67-383 &   13.59 &  8h50m47.69s &   132d41m55.35s &  132.698708333 &   11d47m52.9s &  11.7980277778 \\
   5 &       M67-385 &  14.526 &  8h50m48.02s &   132d42m00.27s &     132.700075 &  11d49m41.49s &  11.8281916667 \\
   6 &       M67-386 &  14.085 &  8h50m48.11s &   132d42m01.65s &  132.700458333 &  11d54m47.66s &  11.9132388889 \\
   7 &       M67-389 &   17.23 &  8h50m50.56s &    132d42m38.4s &  132.710666667 &     11d41m17s &  11.6880555556 \\
   8 &       M67-392